In [1]:
import pandas as pd
import os
import numpy as np

# Import Data

In [2]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [3]:
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsMale              891 non-null    int64  
 6   Deck_A              891 non-null    int64  
 7   Deck_B              891 non-null    int64  
 8   Deck_C              891 non-null    int64  
 9   Deck_D              891 non-null    int64  
 10  Deck_E              891 non-null    int64  
 11  Deck_F              891 non-null    int64  
 12  Deck_G              891 non-null    int64  
 13  Deck_Z              891 non-null    int64  
 14  Pclass_1            891 non-null    int64  
 15  Pclass_2            891 non-null    int64  
 16  Pclass_3

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   FamilySize          418 non-null    int64  
 3   IsMother            418 non-null    int64  
 4   IsMale              418 non-null    int64  
 5   Deck_A              418 non-null    int64  
 6   Deck_B              418 non-null    int64  
 7   Deck_C              418 non-null    int64  
 8   Deck_D              418 non-null    int64  
 9   Deck_E              418 non-null    int64  
 10  Deck_F              418 non-null    int64  
 11  Deck_G              418 non-null    int64  
 12  Deck_Z              418 non-null    int64  
 13  Pclass_1            418 non-null    int64  
 14  Pclass_2            418 non-null    int64  
 15  Pclass_3            418 non-null    int64  
 16  Title

# Data Preperation

In [11]:
X = train_df.loc[:,'Age':].values.astype('float')
y = train_df['Survived'].ravel()

In [12]:
print(X.shape, y.shape)

(891, 32) (891,)


In [13]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [14]:
# average survival in train and test
print('mean survival in train : {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test : {0:.3f}'.format(np.mean(y_test)))

mean survival in train : 0.383
mean survival in test : 0.385


### Check Scikit-Learn Version

In [15]:
import sklearn

In [16]:
sklearn.__version__

'0.23.2'

# Baseline Model

In [17]:
from sklearn.dummy import DummyClassifier

In [18]:
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [19]:
model_dummy.fit(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

In [20]:
print('Score for baseline model : {0:.2f}'.format(model_dummy.score(X_test,y_test)))

Score for baseline model : 0.61


In [21]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [22]:
print('accuracy for baseline model : {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model : 0.61


In [23]:
print('confusion matrix for baseline model: \n {}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [24]:
print('precision for baseline model : {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model: {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model : 0.00
recall for baseline model: 0.00


C:\Users\jimc\.conda\envs\env_dataScience\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# First Kaggle Submission using Baseline Model

In [28]:
test_X = test_df.values.astype('float')

In [30]:
predictions = model_dummy.predict(test_X)

In [31]:
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [32]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [34]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')

In [35]:
df_submission.to_csv(submission_file_path, index=False)

In [36]:
# create function of above cels
def get_submission_file(model, filename):
    test_X = test_df.values.astype('float')
    predictions = model.predict(test_X)
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    df_submission.to_csv(submission_file_path, index=False)

In [37]:
get_submission_file(model_dummy, '01_dummy.csv')

# Logistic Regression Model

In [38]:
from sklearn.linear_model import LogisticRegression

In [43]:
# create model
model_lr_1 = LogisticRegression(random_state=0, max_iter=800)

In [44]:
# train model
model_lr_1.fit(X_train,y_train)

LogisticRegression(max_iter=800, random_state=0)

In [45]:
# evaluate model
print('score for logistic regression - version 1 : {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistic regression - version 1 : 0.83


# Second Kaggle Submission

In [46]:
get_submission_file(model_lr_1, '02_lr.csv')